In [30]:
import requests
import json
import io
import pandas as pd
import numpy as np
import os

Step 1: Get 2010 chi census tracks into a pandas dataFrame \
Step 2: Get 2020 chi census tracks into a pandas dataFrame \
Step 3: Determine relationship between 2010 and 2020. \
Step 4: Limit to just census tracks in 2020 and 2010 that have a one to one relationship

In [2]:
# step 1
def extract_chi_census_tracts_2010(filename):
    """
    Takes in census track data and returns shorted table to filter by
    census track for chicago

    """
    census = pd.read_csv(filename, dtype=str)
    columns = ["TRACTCE10", "GEOID10", "NAME10", "NAMELSAD10"]
    census = pd.read_csv(filename, dtype=str)

    return census[columns]

In [6]:
# step 2
def extract_chi_census_tracts_2020(filename):
    """
    Takes in census track data and returns shorted table to filter by
    census track for chicago
    """
    census = pd.read_csv(filename, dtype=str)

    return census

In [8]:
#all census tracts for chicago from 2010
chi_census_tracts_2010 = extract_chi_census_tracts_2010(
    "raw_data/census_tracts_2010.csv")

#all census tracts for chicago from 2020
chi_census_tracts_2020 = extract_chi_census_tracts_2020(
    "raw_data/chi_ct_2020.csv")

In [34]:
# step 3 / 4
# https://www.census.gov/geographies/reference-files/time-series/geo/relationship-files.2020.html#tract

def census_tracts_2020_2010_relationships(url):
    """
    returns a mapping of only 1:1 census tracts from 2020 and 2010 with GEOID_TRACT 
    """
    data = requests.get(url).content
    relationships = pd.read_csv(io.StringIO(data.decode('utf-8')), sep = "|")

    chi_census_tracts_2010 = extract_chi_census_tracts_2010(
    "raw_data/census_tracts_2010.csv")

    # filter to just tracts in chicago, will need to be 2020 
    chi_GEOID10 = list(chi_census_tracts_2010['GEOID10'].astype(int))
    filter = relationships['GEOID_TRACT_10'].isin(chi_GEOID10)
    chi_relationships = relationships[filter]

    dupe_count = chi_relationships['GEOID_TRACT_10'].value_counts().reset_index()

    chi_relationships_flag = chi_relationships.merge(dupe_count, how='left', 
                                                     left_on='GEOID_TRACT_10', 
                                                     right_on='GEOID_TRACT_10')    

    filter = chi_relationships_flag['count'] == 1 
    chi_relationships_flag['relation'] = np.where(filter, 'one', 'many')

    #limit columns
    columns = ['GEOID_TRACT_20','GEOID_TRACT_10', 'relation']

    return chi_relationships_flag[columns].reset_index(drop=True)

In [ ]:
url = 'https://www2.census.gov/geo/docs/maps-data/data/rel2020/tract/tab20_tract20_tract10_natl.txt'
chi_ct_relationships = census_tracts_2020_2010_relationships(url)